# Quantum Random Number Generator - Simulation


## DV QRNG

### Core Physics 
A **qubit** is a two level quantum system with states $|0\rangle$ and $|1\rangle$.

If we prepare the qubit in the state $|0\rangle$ and apply a **Hadamard gate** $H$, the state becomes: $|+\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}$

If we now measure this state in the **computational basis** $\{ |0\rangle, |1\rangle \}$, the **Born rule** says:
$P(0) = |\langle 0 | + \rangle|^2 = \tfrac{1}{2}$ and $P(1) = |\langle 1 | + \rangle|^2 = \tfrac{1}{2}$

*Intrinsic* randomness of quantum mechanics vs deterministic classical randomness (chaos). 

Even if we had complete information about the system, the measurement outcome is **fundamentally unpredictable**.  
Each measurement therefore gives one *truly random bit*.

#### **Qiskit**
- IBM’s open-source Python framework for quantum computing
- allows the simulation of quantum circuits on a classical computer
- run the same circuits on **real IBM quantum processors** in the cloud

This means that:
- on a simulator, we get *simulated quantum statistics* (generated using a classical pseudo-random number generators)  
- on a real IBM quantum device, we can obtain *genuine quantum random numbers*

$\rightarrow$ i created an IBM Quantum account and i am trying to get cloud access to real quantum chips (working on it...)

<div class="alert alert-block alert-info">
<b>coming in the near future
</div>


In [111]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

In [112]:
qc = QuantumCircuit(1, 1)  
qc.h(0)  
qc.measure(0, 0)

backend = AerSimulator() 
transpiled_qc = transpile(qc, backend)
job = backend.run(transpiled_qc, shots=1000000, memory=True)
result = job.result()
bitstring = np.array(result.get_memory(), dtype=np.uint8)
print('Bitstring: ', bitstring)
print('First 100 bits of Bitstring: \n', bitstring[:100])

counts = result.get_counts()
print()
print('Random bits generated: ', counts)


Bitstring:  [1 1 0 ... 1 0 0]
First 100 bits of Bitstring: 
 [1 1 0 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 0 0
 0 1 1 0 0 0 1 1 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 1
 0 0 0 1 1 0 0 1 1 0 0 0 0 1 0 1 0 0 0 1 1 0 1 1 0 0]

Random bits generated:  {'0': 499624, '1': 500376}


#### **$H_{min}$ *Min-Entropy***  

Min-entropy ($H_{min}$) is a measure of the unpredictability of a random variable. It is defined as: $H_{min}(X) = -log_2(max(P(X=x)))$

- represents the worst case predictability where:
    - 0 $\rightarrow$ fully predictable
    - 1 $\rightarrow$ perfectly random (maximally unpredictable)


In [113]:
def min_entropy(bitstring):
    shots = len(bitstring)
    counts = np.bincount(bitstring, minlength=2)
    probs = counts / shots
    return -np.log2(np.max(probs))

h_min = min_entropy(bitstring)
print('Min Entropy: ', h_min)

Min Entropy:  0.9989155010497685


### **Randomness Extraction Models**

Randomness Extraction Models are methods / algorithms that take biased sources of randomness and produce uniform, unbiased random bits

- Von Neumann Extractor (pair-wise)
- Hash-based Extractor

#### Von Neumann Extractor

The **von Neumann extractor** is a simple method to *remove bias* from a random bit sequence. It works by reading the bits in **pairs** of two:
-  `00` or `11` $\rightarrow$ discard
-  `01` $\rightarrow$ output `0`
-  `10` $\rightarrow$ output `1`

$\Rightarrow$ more **uniformly random sequence**, even if the original source has a slight bias.


In [114]:
def von_neumann_extraction(bits):
    extracted_bits = []
    for i in range(0, len(bits)-1, 2):
        pair = bits[i], bits[i+1]
        if pair == (0, 1):
            extracted_bits.append(0)
        elif pair == (1, 0):
            extracted_bits.append(1)
    return np.array(extracted_bits, dtype=np.uint8)

vn_bits = von_neumann_extraction(bitstring)
print('Initial bitstring length: ', len(bitstring))
print('Extracted (Von Neumann) bitstring length: ', len(vn_bits))
percent_survived = (len(vn_bits) / len(bitstring)) * 100
print(f'Percentage of bits that survived: {percent_survived:.2f}%')

unique, counts_vn = np.unique(vn_bits, return_counts=True)
counts_dict = {int(k): int(v) for k, v in zip(unique, counts_vn)}
print()
print('Initial counts: ', counts)
print('Von Neumann counts: ', counts_dict)

shots_vn = len(vn_bits)
if shots_vn > 0:
    probs_vn = counts_vn / shots_vn
    h_min_vn = -np.log2(np.max(probs_vn))
    print()
    print('Initial Min Entropy: ', h_min)
    print('Min Entropy after Von Neumann extraction: ', h_min_vn)
else:
    print('No bits survived the Von Neumann extraction')



Initial bitstring length:  1000000
Extracted (Von Neumann) bitstring length:  250458
Percentage of bits that survived: 25.05%

Initial counts:  {'0': 499624, '1': 500376}
Von Neumann counts:  {0: 125275, 1: 125183}

Initial Min Entropy:  0.9989155010497685
Min Entropy after Von Neumann extraction:  0.999470156383716


#### Von Neumann extraction:
- pros: nice & simple method 

- cons: discards a lotttt of bits (~75%) $\Rightarrow$ wastes data

     - computational cost (extra memory & CPU cycles)

    - time inefficiency 

### Hash-based Extraction 

Hash functions = math function / algorithm that maps input data of any size to a fixed-size string output
- cryptographic hash functions (SHA-256, SHA-3, Blake2, MD5) $\rightarrow$ less secure now


*Hash-based Randomness Extraction model* uses hash functions as extractors bc they:

- get a biased random input & they output a uniform(-ish?) bitstring 

- retain nearly all entropy from the input without discarding most bits (unlike Von Neumann)


<div class="alert alert-block alert-info">
<b>coming in the near future
</div>

so far ideal simulator, thus $\Rightarrow$ simulating realistic quantum noise

### **Adding Noise** yay

*Qiskit's Noise Models* $\Rightarrow$ pre-defined noise models from *real IBM quantum devices* (+ custom) can simulate:

- gate errors (hadamard)

- readout / measurement errors 

- decoherence errors (T1/T2 relaxation) aka qubit loses its quantum information over time due to interactions with the environment
    - T1 relaxation (energy relaxation) $\rightarrow$ qubit loses energy to its environment (from $|0\rangle$ to $|1\rangle$)

        $\Rightarrow$ flips to $|0\rangle$ over time, thus: prob of remaining in $|1\rangle$ decreases as: $e^{-t/T_1}$
    
    - T2 relaxation (dephasing) (no energy change) $\rightarrow$ loss of phase information in a superposition
     
        $\Rightarrow$ qubit’s superposition collapses faster than intended, thus coherence decays as: $e^{-t/T_2}$





In [115]:
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError

noise_model = NoiseModel()

gate_error = depolarizing_error(0.01, 1)  # applying a 1-qubit gate (Hadamard) => 1% chance that the qubit gets randomized
noise_model.add_all_qubit_quantum_error(gate_error, ['h'])

readout_error = ReadoutError([[0.98, 0.02], [0.03, 0.97]])  # confusion matrix => 98% for |0⟩ 2% for |1⟩, and 97% for |1⟩ 3% for |0⟩
noise_model.add_all_qubit_readout_error(readout_error)  # favouring the |0⟩ state

'''
# readout error just for dark counts, confusion matrix:
ReadoutError([[0.99, 0.01],  # true 0 (99%) => sometimes mistaken as 1 (dark count)
              [0.00, 1.00]]) # true 1 (100%) always read correctly
'''

backend_noisy = AerSimulator(noise_model=noise_model)
transpiled_qc_noisy = transpile(qc, backend_noisy)
job_noisy = backend_noisy.run(transpiled_qc_noisy, shots=100000, memory=True)
result_noisy = job_noisy.result()
bitstring_noisy = np.array(result_noisy.get_memory(), dtype=np.uint8)

print('Noisy bitstring sample: \n', bitstring_noisy[:100])

h_min_noisy = min_entropy(bitstring_noisy)
print()
print('Min Entropy (ideal system): ', min_entropy(bitstring))
print('Noisy Min Entropy: ', h_min_noisy)

Noisy bitstring sample: 
 [0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 0 1
 1 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 1 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 0 1 0 0
 1 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0 1 1 1 1 1 1 1 0 0 1]

Min Entropy (ideal system):  0.9989155010497685
Noisy Min Entropy:  0.9806823810378297


## CV QRNG

### Core Physics 

#### (1) Principle of CV QRNGs

CV QRNG use **intrinsic quantum fluctuations** of a physical observable, typically the **quadratures of the electromagnetic field** in an optical mode. Unlike DV QRNG (single-photon detection), CV QRNG measure a **continuous distribution**.

- Key idea: **Quantum vacuum noise** or **shot noise** provides a fundamentally random signal.  
- Observable: quadrature $ \hat{X} $ of a light field.

$\hat{X} = \frac{1}{2} (\hat{a} + \hat{a}^\dagger)$ where $\hat{a}$ and $\hat{a}^\dagger$ are the annihilation and creation operators.

#### (2) Quantum Noise Source

The **vacuum state** $|0\rangle$ exhibits zero-point fluctuations:

$\langle \hat{X} \rangle = 0, \quad \Delta X = \sqrt{\langle \hat{X}^2 \rangle - \langle \hat{X} \rangle^2} = \frac{1}{2}$

- This variance $ \Delta X^2 $ represents **irreducible quantum uncertainty**.  
- Measuring the quadrature produces outcomes distributed according to a **Gaussian probability density function (PDF)**.

$P(X) = \frac{1}{\sqrt{\pi}} e^{-X^2}$

#### (3) Measurement and Digitization

1. The optical signal is measured via **homodyne detection**:
   - Interference with a strong local oscillator converts the quadrature to a measurable voltage.
2. The continuous outcome $ X $ is **sampled and digitized** to produce raw bits.  

- Typically, the voltage is split into bins to form discrete random numbers.
- Raw distribution is **Gaussian**, not uniform. Post-processing or randomness extraction is needed for uniform bits.

#### (4) Randomness Extraction

- To convert Gaussian-distributed raw data to uniform random bits:
  - **Von Neumann extraction**: removes bias in pairs of bits.
  - **Hash functions / Trevisan extractors**: extract almost uniform bits from a high-entropy source.

- Min-entropy is used to **quantify the worst-case unpredictability** of the raw measurement.

$H_{\min} = -\log_2 \max_{x} P(x)$

#### 5. Advantages of CV QRNGs

- High-speed generation (GHz rates possible) because of **continuous measurement**.  
- Can leverage **standard photodiodes** — no single-photon detectors needed.  
- Fundamentally quantum: unpredictability guaranteed by **Heisenberg uncertainty principle**.


